In [ ]:
import rasterio
from rasterio import plot
import matplotlib.pyplot as plt
import numpy as np


In [ ]:
import yaml
config_path = '/home/tu/tu_tu/tu_zxmav84/DS_Project/modules/config.yml'
with open(config_path, 'r') as f:
    config = yaml.load(f, Loader=yaml.FullLoader)

landsat_path = config['data']['landsat']
config

In [ ]:
base_path = landsat_path+'/raw_scenes/LC08_L1TP_193026_20220803_20220806_02_T1/LC08_L1TP_193026_20220803_20220806_02_T1_'

In [ ]:
#import bands as separate 1 band raster
band4 = rasterio.open(base_path+'B4.tif') #red
band5 = rasterio.open(base_path+'B5.tif') #nir

band5.height, band5.width

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))
plot.show(band4, ax=ax1, cmap='Blues') #red
plot.show(band5, ax=ax2, cmap='Blues') #nir
fig.tight_layout()

In [ ]:
#generate nir and red objects as arrays in float64 format
red = band4.read(1).astype('float64')
nir = band5.read(1).astype('float64')
nir

In [ ]:
#ndvi calculation, empty cells or nodata cells are reported as 0
ndvi=np.where(
    (nir+red)==0., 
    0, 
    (nir-red)/(nir+red))
ndvi[:5,:5]

In [ ]:
#export ndvi image
ndviImage = rasterio.open('ndviImage.tif','w',driver='Gtiff',
                          width=band4.width, 
                          height = band4.height, 
                          count=1, crs=band4.crs, 
                          transform=band4.transform, 
                          dtype='float64')
ndviImage.write(ndvi,1)
ndviImage.close()

In [ ]:
#plot ndvi
ndvi = rasterio.open('ndviImage.tif')
fig = plt.figure(figsize=(18,12))
plot.show(ndvi)

### Subset to Munich

In [ ]:
bbox_4326 = (11.576947,48.128678,11.604069,48.149555)
import rasterio.warp
[xmin, xmax], [ymin, ymax] = rasterio.warp.transform('EPSG:4326', band4.crs, [bbox_4326[0], bbox_4326[2]], [bbox_4326[1], bbox_4326[3]])
bbox = (xmin, ymin, xmax, ymax)
bbox

In [ ]:
from rasterio.windows import from_bounds
with rasterio.open('ndviImage.tif') as src:
    # Get the window corresponding to the bounding box
    window = from_bounds(*bbox, src.transform)

    # Read the subset of the raster data as a numpy array
    subset = src.read(window=window)

    # Update the metadata of the new GeoTIFF to reflect the subset
    meta = src.meta
    meta.update({
        'width': window.width,
        'height': window.height,
        'transform': rasterio.windows.transform(window, src.transform)
    })

# Write the subset of the raster data to a new GeoTIFF file
with rasterio.open('my_subset.tif', 'w', **meta) as dst:
   dst.write(subset)
fig, ax = plt.subplots(figsize=(12, 6))
plt.imshow(subset[0])